In [1]:
import numpy as np
import re
import matplotlib.pyplot as plt
import sympy as syp
from sympy import Q
from sympy.assumptions.refine import refine_sign
from IPython.display import HTML 

def pprint(expr):
    latex = syp.latex(expr)
    latex = re.sub(r"\\operatorname", "", latex)
    display(HTML(r"${}$".format(latex)))

# Deriving gradients for active inference in Continuous Time

Case with:
* one latent variable (the current belief about the angle position) 
* two observation dimensions (proprioceptive and visual)


### Names


* $\boldsymbol{\mu}$ - central value of the internal latent variable (belief about the generative process latent variable)   
* $\boldsymbol{\mu}$' - increment of $\mu$ 
* $\boldsymbol{\rho}$ - desired state of the generative process latent variable 
* $\mathbf{a}$ - action done by the agent 
* $\mathbf{s_{p}(a)}$ - proprioceptive observation (**as a function of the action**) 
* $\mathbf{s_{v}(a)}$ - visual observation (**as a function of the action**)   
* $\mathbf{g(\mu)}$ - Forward model that predicts the visual observations based on the latent variable. (In this case the proprioceptive observation is in the same domain as the latent variable, thus thre is no need of a forward model for it)
* $\mathbf{f(\mu, \rho)}$ - Model of the dynamics of the generative controlling the response to the agent's action. The model's dynamics is biased so that it has $\rho$ as an attractor, so that the agent's belief leads to a real movement towards $\rho$ 

In [2]:
x, C,a, mu, mu1, rho = syp.symbols(r"x C a \mu \mu' \rho")
sigma_sp, sigma_sv, sigma_mu = syp.symbols(r'\Sigma_{s_p} \Sigma_{s_v} \Sigma_{mu}')

g = syp.Function("g")(mu)
f = syp.Function("f")(mu, rho)
sp = syp.Function(r"s_p")(a)
sv = syp.Function(r"s_v")(a)

def normal(x, m, s): 
    return (1/(s*syp.sqrt(2*syp.pi)))*syp.exp(-((x - m)**2)/(2*s**2))

### Variational Laplace Encoded Free Energy

In [3]:
p_sp_mu = normal(sp, mu, sigma_sp)                                          
p_sv_mu = normal(sv, g, sigma_sv)
p_mu1_mu_rho = normal(mu1, f, sigma_mu)


F = -syp.log(p_sp_mu*p_sv_mu*p_mu1_mu_rho) + C

---
---
$F = -log(P(s_p |\mu)P(s_v | g(\mu))P(\mu'| \mu, \rho)) + C$

In [4]:
pprint(F)

### Gradients

In [5]:
dmu = syp.diff(F, mu)
dmu = syp.simplify(dmu)
dmu = syp.collect(dmu, sigma_mu)
dmu = syp.collect(dmu,syp.diff(f, mu))
dmu = syp.collect(dmu, sigma_sv)
dmu = syp.collect(dmu, sigma_sp)
dmu = syp.collect(dmu,syp.diff(g, mu))
pprint(syp.Eq(-syp.Derivative("F", mu), -dmu))

In [6]:
dmu1 = syp.diff(F, mu1)
dmu1 = syp.simplify(dmu1)
-syp.simplify(dmu1)
pprint(syp.Eq(-syp.Derivative("F", mu1),-syp.simplify(dmu1)))

In [7]:
da = syp.simplify(syp.diff(F, a))
da = syp.collect(da, sigma_sp)
da = syp.collect(da, syp.diff(sp, a))
da = syp.collect(da, sigma_sv)
da = syp.collect(da, syp.diff(sv, a))
pprint(syp.Eq(-syp.Derivative("F", a),-da))

### World model dynamics

The dynamics are those of an harmonic oscillator:

In [8]:
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
init_vprinting()

a, k, phi, z, t, muc = syp.symbols("a k phi z t mu")
f  = syp.Function("f")(t)

fmu = f.diff(t,t) + k*f + phi*f.diff(t) - a
pprint(syp.Eq(fmu, 0))


whose solution is an equation of type:

In [9]:
h = syp.dsolve(fmu, f)
pprint(h)

In our case $f(t, a)$ indicates the proposed value of $\mu$ given an
action ($a$).

The prediction about $s_p(a, t)$ is then:

$s_p(a, t) = f + \epsilon$

with $\epsilon \in \mathcal{N}(0, \Sigma_{s_p})$

That is:

In [10]:
eps = syp.symbols("epsilon")
sp_pred = h.subs(a/k, a/k + eps)
pprint(syp.Eq(sp, sp_pred.rhs))

and its derivative w.r.t. $a$ is:

In [11]:
pprint(syp.Eq(syp.diff(sp, a), syp.diff(sp_pred.rhs,a)))

And the same goes for the visual observation

In [12]:
pprint(syp.Eq(syp.diff(sv, a), syp.diff(sp_pred.rhs,a)))